## Config

In [1]:
from scipy.optimize import fsolve
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import numpy as np
import numpy as np
import pandas as pd
from openpyxl import Workbook
from datetime import datetime
import os
import ntpath

np.random.seed(42)

m = 20  #                   number of EVSEs
n = 20  #                   time periods in the future (can become less)
r = 0.5  #                  factor of Enexis grid connection

# --------------------------------------------------------------------------
# The array's for all EV's & SE together and the resulting minimum.
# EV = np.random.choice([EV_MPI, EV_MPI / 2, 3.33, 3.33 / 2], size=(m, n))
# SE = np.random.choice([SE_MPO], size=(m, n))
# EVSE = np.minimum(EV, SE)


SE_MPO = 7  #               EVSE max Power output for each time period
EV_MPI = 7  #               EV max Power input for each time period
EX_MPO = m * r * SE_MPO  #  enexis max Power output for each time period (constant)
CAP = 70  #                 EV max capacity of battery
K = 0.075  #                EV battery degradation cost per kWh
# --------------------------------------------------------------------------

soc_l = 0.0  #              lower bound of state of charge
soc_h = 0.2  #              upper bound of state of charge
d_c_l = 0.4  #              lower bound of desired charge
d_c_h = 1.0  #              upper bound of desired charge
dur_l = 1.0  #              lower bound of parking duration
dur_h = 8.0  #              upper bound of parking duration
eng_l = 0.3  #              lower bound of energy price
eng_h = 0.6  #              upper bound of energy price
p_rnd = True  #             will energyprice be random of linear

alpha = 1.0  #              EVSE efficiency
beta = 1.0  #               customer satisfaction
gamma = 1.0  #              cost of energy

# --------------------------------------------------------------------------
print_solver_outcome = False
print_EVSE_power = False

print_session_max_charge = False
print_remaining_charge = False

print_parking_time = False
print_energy_price = False

print_tee = False

# Set the solver to be used
# solver = "ipopt" #        m 50 EVSE's, > 50 time periods
# solver = "mosek" #        m 50 EVSE's, > 50 time periods
# solver = "cplex"  #       m 50 EVSE's, 18 time periods
solver = "glpk"  #          m 50 EVSE's, > 50 time periods
# solver = "gurobi" #       m xx EVSE's, 19 time periods - no license yet

## Functions

### EV Charge Profile

In [2]:
def cv_pwr(t: float, pm: float, k: float) -> float:
    """
    Calculate the charging power for a given time.

    Parameters
    ----------
    t : float
        Time in hours.
    pm : float
        Maximum power input of the EV in kW.
    k : float
        Charging curve constant.
        0.01-0.03 charge aggressively,
        0.05-0.1  prioritizing battery health and longevity
    """
    return pm * np.exp(-k * t)


def cv_eng(t2, t1, pm, k) -> float:
    """
    Calculate the charging energy for a given time interval (t2, t1)
    note: t2 > t1

    Parameters
    ----------
    t2 : float
        End time in hours.
    t1 : float
        Start time in hours.
    pm : float
        Maximum power input of the EV in kW.
    k : float
        Charging curve constant.
        0.01-0.03 charge aggressively,
        0.05-0.1  prioritizing battery health and longevity
    """
    return (-1 / k) * (cv_pwr(t2, pm, k) - cv_pwr(t1, pm, k))


# def cv_pwr_avg(t2, t1, pm, k) -> float:
#     return cv_eng(t2, t1, pm, k) / (t2 - t1)


def charge_profile(
    dur: float,
    soc: float,
    d_c: float,
    cap: float = CAP,
    ev_mpi: float = EV_MPI,
    se_mpo: float = SE_MPO,
    k: float = K,
) -> list:
    """
    Calculate the charging profile for a given EV.

    Parameters
    ----------
    dur : float
        Duration of stay in hours.
    soc : float
        State of charge of the battery in %.
    d_c : float
        Desired charge of the battery in %.
    cap : float
        Capacity of the battery in kWh.
    ev_mpi : float
        Maximum power input of the EV in kW.
    se_mpo : float
        Maximum power output of the EVSE in kW.
    k : float
        Charging curve constant.
        0.01-0.03 charge aggressively,
        0.05-0.1  prioritizing battery health and longevity

    Returns
    -------
    list
        List of the charging profile in the format [time, power, charge].
    """
    # --------------------------------------------------------------------------
    # Calculate the charging profile
    # https://www.homechargingstations.com/ev-charging-time-calculator/
    # --------------------------------------------------------------------------
    # Calculate the charge required to reach the desired capacity
    cv = 0.8 * cap  #                                charging curve flattens at cap.
    dc = d_c * cap  #                                soc + charge [kWh] required
    mp = min(ev_mpi, se_mpo)  #                      max power [kW] for charging

    # current charge
    uc0 = soc * cap  #                                charge [kWh] at entry
    up0 = mp  #                                       max power [kW] at entry
    ut0 = uc0 / up0  #                                time required for charge at entry

    # first part of charge < 80% of cap
    uc1 = max(min(dc, cv) - min(uc0, cv), 0)  #      charge [kWh] required
    up1 = 0 if uc1 == 0 else mp  #                   max power [kW] for charging
    ut1 = 0 if uc1 == 0 else uc1 / up1  #            time [h] required
    # constrainted by duration
    ct1 = min(dur, ut1)  #                           time constrainted by duration
    cp1 = up1  #                                     max power [kW] for charging
    cc1 = ct1 * cp1  #                               charge constrainted by duration

    # second part of charge > 80% of cap
    uc2 = max(max(dc, cv) - max(uc0, cv), 0)  #      charge [kWh] required
    up2 = 0  #                                       initial power [kW] required
    ut2 = 0  #                                       initial time [h] required
    ct2 = 0  #                                       time constrainted by duration
    cp2 = 0  #                                       max power [kW] for charging
    cc2 = 0  #                                       charge constrainted by duration

    # Define the function for the given equation with specific ta, pm and k
    def zero_for_E(t2, t1, pm, k, E) -> float:
        """
        function to solve the root of the equation, so where it is 0

        Parameters
        ----------
        t2 : float
            End time in hours.
        t1 : float
            Start time in hours.
        pm : float
            Maximum power input of the EV in kW.
        k : float
            Charging curve constant.
            0.01-0.03 charge aggressively,
            0.05-0.1  prioritizing battery health and longevity
        E : float
            Energy in kWh to be charged
        """
        return cv_eng(t2, t1, pm, k) - E

    # determime time and power for 2nd part of charge CV
    if uc2 > 0:
        # Solve the equation numerically
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html
        solution = fsolve(func=zero_for_E, x0=ut1, args=(0, mp, k, uc2), xtol=1e-3)

        ut2 = solution[0]  #                         time [h] required during CV
        up2 = cv_pwr(ut2, mp, k)  #                  power [kW] required during CV

        ct2 = max(0, min(dur - ut1, ut2))  #         real time [h] during CV

        if ct2 > 0:
            cc2 = cv_eng(ct2, 0, mp, k)  #           charge [kWh] during CV
            cp2 = cv_pwr(ct2, mp, k)  #              real power [kW] during CV

    # --------------------------------------------------------------------------
    e2 = 0 if ut2 == 0 else float(zero_for_E(t2=ut2, t1=0, pm=mp, k=k, E=uc2))

    ut3 = max(0, dur - (ut1 + ut2))  #                slack time [h] for parking

    return {
        "params": {
            "dur": dur,
            "soc": soc,
            "d_c": d_c,
            "cap": cap,
            "mxp": mp,
            "k": k,
        },
        "phase0": {"c0": uc0, "t": ut0, "p": up0},
        "phase1": {"c1": uc1, "t": ut1, "p": up1},
        "real_1": {"c1": cc1, "t": ct1 + ut3, "p": cp1},
        "phase2": {"c2": uc2, "t": ut1 + ut2, "p": up2},
        "real_2": {"c2": cc2, "t": ct1 + ct2 + ut3, "p": cp2},
        "result": {
            "ufc": uc0 + uc1 + uc2 + e2,
            "cfc": uc0 + cc1 + cc2 + e2,
            "dc": dc,
            "rm": dc - (uc0 + uc1 + uc2 + e2),
        },
        "tslots": {"t1": ut1, "T1": ct1, "t2": ut2, "T2": ct2, "t3": ut3, "T3": ut3},
    }

### Battery Charge kWh

$$
\int_{a}^{b} P_{\text{max}} \times e^{-k \times (t - t_0)} \, dt = 
-\frac{P_{\text{max}}}{k} \left[ e^{-k \times (b - t_0)} - e^{-k \times (a - t_0)} \right]

$$


### OLP - Abstract Model

In [3]:
def create_model_TGC(
    m: int = m,
    n: int = n,
    alpha: float = alpha,
    beta: float = beta,
    gamma: float = gamma,
) -> pyo.ConcreteModel:
    """
    Create the TGC model.

    Parameters
    ----------
    m : int
        Number of EVSEs.
    n : int
        Number of time periods.
    alpha : float
        EVSE efficiency.
    beta : float
        Customer satisfaction.
    gamma : float
        Cost of energy.

    Returns
    -------
    pyo.ConcreteModel
        The model object.
    """
    # --------------------------------------------------------------------------
    # TGC: Tetris Game Charger
    # --------------------------------------------------------------------------

    # --------------------------------------------------------------------------
    # Abstract Models
    # https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
    # --------------------------------------------------------------------------

    TGC = pyo.AbstractModel()

    # --------------------------------------------------------------------------
    # Sets
    # https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
    # note: pyomo uses 1-based indexing, these sets are fixed
    # the first value and step size default to 1
    # --------------------------------------------------------------------------

    TGC.I = pyo.RangeSet(m, doc="fixed set of EVSEs")
    TGC.J = pyo.RangeSet(n, doc="fixed set of time periods for a certain horizon (h=5)")

    # --------------------------------------------------------------------------
    # Parameters
    # https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
    # note: pyomo uses 1-based indexing, these parameters are all mutable
    # --------------------------------------------------------------------------

    # energy price per kWh for each time period
    TGC.energy_price = pyo.Param(TGC.J, default=0, mutable=True, within=pyo.Reals)

    # enexis max Power output for each time period (constant)
    TGC.enexis = pyo.Param(TGC.J, default=0, mutable=True, within=pyo.NonNegativeReals)

    # max Power EVSE session for each time period
    TGC.session = pyo.Param(
        TGC.I, TGC.J, default=0, mutable=True, within=pyo.NonNegativeReals
    )

    # remaining charge for each session
    TGC.remaining_charge = pyo.Param(
        TGC.I, default=0, mutable=True, within=pyo.NonNegativeReals
    )

    # time period length for each time period
    TGC.pt = pyo.Param(TGC.J, default=0, mutable=True, within=pyo.NonNegativeReals)

    # weight for each time period
    TGC.w = pyo.Param(TGC.J, default=0, mutable=True, within=pyo.NonNegativeReals)

    # --------------------------------------------------------------------------
    # Variables
    # https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
    # --------------------------------------------------------------------------

    TGC.x = pyo.Var(TGC.I, TGC.J, domain=pyo.NonNegativeReals)

    # --------------------------------------------------------------------------
    # Objective function
    # https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
    # --------------------------------------------------------------------------

    def obj_expression(TGC) -> float:
        """
        Objective function for the TGC problem.

        Parameters
        ----------
        model : pyomo.environ.ConcreteModel
            The model object.

        Returns
        -------
        float
            The objective value.
        """

        return (
            alpha
            * sum(
                TGC.w[j] * (1 - sum(TGC.x[i, j] for i in TGC.I) / TGC.enexis[j])
                for j in TGC.J
            )
            + beta
            * sum(
                (
                    1
                    - sum(TGC.pt[j] * TGC.x[i, j] for j in TGC.J)
                    / TGC.remaining_charge[i]
                )
                for i in TGC.I
            )
            / m
            + (
                gamma
                / sum(TGC.energy_price[j] * TGC.pt[j] * TGC.enexis[j] for j in TGC.J)
            )
            * sum(
                (TGC.energy_price[j] * TGC.pt[j] * sum(TGC.x[i, j] for i in TGC.I))
                for j in TGC.J
            )
        )

    TGC.OBJ = pyo.Objective(rule=obj_expression)

    # --------------------------------------------------------------------------
    # Constraints
    # https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
    # https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
    # --------------------------------------------------------------------------

    def ex_grid_constraint_rule(TGC, j) -> float:
        """
        Constraint: Total power of all EVSEs in a time period (j)
        must be less than or equal to the maximum power output of the grid.

        Parameters
        ----------
        model : pyomo.environ.ConcreteModel
            The model object.
        j : int
            The time period.
        """
        return sum(TGC.x[i, j] for i in TGC.I) <= TGC.enexis[j]

    def session_constraint_rule(TGC, i, j) -> float:
        """
        Constraint: Power of EVSE (i) in a time period (j)
        must be less than or equal to the maximum power input of the EVSE.

        Parameters
        ----------
        model : pyomo.environ.ConcreteModel
            The model object.
        i : int
            The EVSE.
        j : int
            The time period.
        """
        return TGC.x[i, j] <= TGC.session[i, j]

    def deschrg_constraint_rule(TGC, i) -> float:
        """
        Constraint: Final charge of EVSE (i) must be less than or equal to
        the desired (realistic) charge of the EVSE.

        Parameters
        ----------
        model : pyomo.environ.ConcreteModel
            The model object.
        i : int
            The EVSE.
        """
        return sum(TGC.pt[j] * TGC.x[i, j] for j in TGC.J) <= TGC.remaining_charge[i]

    # the next line creates one constraint for each member of the set model.J
    TGC.Ex_Grid_Constraint = pyo.Constraint(TGC.J, rule=ex_grid_constraint_rule)
    TGC.DesChrg_Constraint = pyo.Constraint(TGC.I, rule=deschrg_constraint_rule)
    TGC.Session_Constraint = pyo.Constraint(TGC.I, TGC.J, rule=session_constraint_rule)

    # --------------------------------------------------------------------------
    # create a model instance
    # https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
    # https://link.springer.com/book/10.1007/978-3-030-68928-5
    # --------------------------------------------------------------------------

    return TGC.create_instance()


def set_tgc_energy_price(tgc, EnergyPrice):
    """
    Set the energy price for each time period.

    Parameters
    ----------
    tgc : pyomo.environ.ConcreteModel
        The model object.
    EnergyPrice : list
        List of energy prices for each time period.
    """
    for j in tgc.J:
        tgc.energy_price[j].value = EnergyPrice[j - 1]


def set_tgc_session_max_power(tgc, EVSE):
    """
    Set the maximum power for each EVSE for each time period.

    Parameters
    ----------
    tgc : pyomo.environ.ConcreteModel
        The model object.
    EVSE : list
        List of maximum power for each EVSE for each time period.
    """
    for i in tgc.I:
        for j in tgc.J:
            tgc.session[i, j].value = EVSE[i - 1, j - 1]


def set_tgc_remaining_charge(tgc, remaining_charge):
    """
    Set the remaining charge for each EVSE.

    Parameters
    ----------
    tgc : pyomo.environ.ConcreteModel
        The model object.
    remaining_charge : list
        List of remaining charge for each EVSE.
    """
    for i in tgc.I:
        tgc.remaining_charge[i].value = remaining_charge[i - 1]


def set_tgc_enexis_max_power(tgc, EX_MPO):
    """
    Set the maximum power for the Enexis grid for each time period.

    Parameters
    ----------
    tgc : pyomo.environ.ConcreteModel
        The model object.
    EX_MPO : float
        Maximum power for the Enexis grid for each time period.
    """
    for j in tgc.J:
        tgc.enexis[j].value = EX_MPO


def set_tgc_pt(tgc, pt):
    """
    Set the time period length for each time period and the weights.

    Parameters
    ----------
    tgc : pyomo.environ.ConcreteModel
        The model object.
    pt : list
        List of time period length for each time period.
    """
    for j in tgc.J:
        tgc.pt[j].value = pt[j - 1]
        tgc.w[j].value = pt[j - 1] / np.sum(pt)

## TGC: Central Controller

### Data collection

In [4]:
# read data from ev
cps = [] # needed for reporting, which should come from ev
data = []
remaining_charge = [] # needed in the TGC model

# from central controler calling ev.get_charge_profile()
for i in range(1, m + 1):
    cp = charge_profile(
        dur=np.random.uniform(dur_l, dur_h), # init of EV
        soc=np.random.uniform(soc_l, soc_h), # init of EV
        d_c=np.random.uniform(d_c_l, d_c_h), # init of EV
        cap=CAP,# init of EV
        ev_mpi=EV_MPI, # init of EV
        se_mpo=SE_MPO, # init of EVSE
        k=K, # init of EV
    )
    cps.append(cp)
    ev = "ev" + str(i).zfill(2)
    data.append({"ev": ev, "t": cp["real_1"]["t"], "p": cp["real_1"]["p"]})
    data.append({"ev": ev, "t": cp["real_2"]["t"], "p": cp["real_2"]["p"]})

    remaining_charge.append(cp["result"]["cfc"] - cp["phase0"]["c0"])

    # data.append({"ev": ev, "t": cp["real_3"]["t"], "p": cp["real_3"]["p"]})
    # print(ev, cp)
    # if i==1:
    #     print(cp["params"])
    #     print(cp["phase0"])
    #     print(cp["phase1"])
    #     print(cp["real_1"])
    #     print(cp["phase2"])
    #     print(cp["real_2"])
    #     print(cp["result"])
    #     print(cp["tslots"])

# --------------------------------------------------------------------------
    
# Create a DataFrame from the data
df = pd.DataFrame(data)

# Remove all records where 't' is 0
df = df.loc[df["t"] != 0]

# df_pivot = df.pivot(index='ev', columns='t', values='p')
df_pivot = df.pivot_table(index="ev", columns="t", values="p", aggfunc="sum")

# Replace NaN values in the last column
df_pivot.iloc[:, -1] = df_pivot.iloc[:, -1].fillna(0)  # ev_mpi


# Replace NaN values in the other columns with the last non-NaN value
def fill_na_with_last_val(row):
    last_val = None
    for col in reversed(row.index):
        if pd.isna(row[col]):
            if last_val is not None:
                row[col] = last_val
        else:
            last_val = row[col]
    return row


df_pivot = df_pivot.apply(fill_na_with_last_val, axis=1)

# Keep only the first X columns or fewer if the DataFrame has fewer than X columns
df_pivot = df_pivot.iloc[:, : min(n, df_pivot.shape[1])]

# ------------------------------------------------------------------------------
# print("\n")
# print(df_pivot)

# Convert the DataFrame to a NumPy array
EVSE = df_pivot.values

if print_EVSE_power:
    print(f"\nEVSE:\n {EVSE}\n")

#### pt

In [5]:
absolute_times = df_pivot.columns.tolist()

pt = [j - i for i, j in zip(absolute_times[:-1], absolute_times[1:])]
pt.insert(0, absolute_times[0])

n = len(pt)  # number of time periods can become less when they overlap
# print(len(pt))

if print_parking_time:
    print(f"\nparking time:\n {pt}\n")


### energy price

In [6]:
# energy price in euro/kWh for each time period
# EnergyPrice = np.array([0.04, 0.08, 0.31, 0.28, 0.12])

if p_rnd:
    EnergyPrice = np.random.uniform(eng_l, eng_h, n)
else:
    EnergyPrice = np.linspace(eng_l, eng_h, n)

if print_energy_price:
    print(f"\nenergy_price:\n {EnergyPrice}\n")

### Instance and Optimize

In [7]:
tgc = create_model_TGC()

# --------------------------------------------------------------------------
# set the parameters
# --------------------------------------------------------------------------

set_tgc_energy_price(tgc, EnergyPrice)
# tgc.energy_price.pprint()

set_tgc_session_max_power(tgc, EVSE)
# tgc.session.pprint()

set_tgc_remaining_charge(tgc, remaining_charge)
# tgc.remaining_charge.pprint()

set_tgc_enexis_max_power(tgc, EX_MPO)
# tgc.enexis.pprint()

set_tgc_pt(tgc, pt)
# tgc.pt.pprint()

# --------------------------------------------------------------------------
# Create a solver
# --------------------------------------------------------------------------
opt = pyo.SolverFactory(solver)
results = opt.solve(tgc, tee=print_tee)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
     print ("OPTIMAL LP SOLUTION FOUND")
elif results.solver.termination_condition == TerminationCondition.infeasible:
     print ("do something about it? or exit?")
else:
     # something else is wrong
     print (str(results.solver))

# outcome of solver model
if print_solver_outcome:
   print(f"\nsolver outcome:\n {tgc.pprint()}\n")


OPTIMAL LP SOLUTION FOUND


## Results

 ### DataFrames

In [8]:
# function(s) to create a DataFrame with the results
def create_df_evse_cs_results():
    tmp = []
    for i in range(1, m + 1):
        tmp.append(
            {
                "EVSE": "EVSE" + str(i).zfill(2),
                "STAY": round(cps[i - 1]["params"]["dur"], 2),
                "SC%": round(100 * cps[i - 1]["phase0"]["c0"] / CAP, 2),
                "DC%": round(100 * cps[i-1]["result"]["dc"] / CAP, 2),
                "DC": round(cps[i-1]["result"]["dc"], 2),                
                "RC": round(cps[i-1]["result"]["cfc"], 2),
                "FC": round(
                    sum(pt[x - 1] * tgc.x[i, x].value for x in range(1, n + 1))
                    + cps[i - 1]["phase0"]["c0"],
                    2,
                ),
                "CS": round(
                    100
                    * (
                        sum(pt[x - 1] * tgc.x[i, x].value for x in range(1, n + 1))
                        + cps[i - 1]["phase0"]["c0"]
                    )
                    / cps[i-1]["result"]["cfc"],
                    2,
                ),
                "ENG0": cps[i - 1]["phase0"]["c0"],
            }
        )
    return pd.DataFrame(tmp)


def create_df_evse_charge_results():
    tmp = []
    for i in range(1, m + 1):
        for x in range(1, n + 1):
            tmp.append(
                {
                    "EVSE": "EVSE" + str(i).zfill(2),
                    "TIME": "T"
                    + str(x).zfill(2)
                    + "_"
                    + str(round(pt[x - 1], 2)).ljust(4, "0"),
                    "POWR": tgc.x[i, x].value,
                    "ENG+": pt[x - 1] * tgc.x[i, x].value,
                    "COST": EnergyPrice[x - 1] * pt[x - 1] * tgc.x[i, x].value,
                }
            )
    return pd.DataFrame(tmp)


df_evse_cs_results = create_df_evse_cs_results()
print(f"\nEVSE_CS:\n{df_evse_cs_results}")

# Overall most detailed results in long format
df_evse_charge_results = create_df_evse_charge_results()
# print(f"\nEVSE_charge:\n{df_evse_charge_results}")

# Overall most detailed results in wide format
df_evse_charge_results_melted = df_evse_charge_results.melt(
    id_vars=["EVSE", "TIME"],
    value_vars=["POWR", "ENG+", "COST"],
    var_name="TYPE",
    value_name="VALUE",
)

dfMTRX = df_evse_charge_results_melted.pivot(
    index=["EVSE", "TYPE"], columns="TIME", values="VALUE"
).reset_index()

# Aggregated results per EVSE
df_evse_charge_results_tot = (
    df_evse_charge_results_melted.groupby(["EVSE", "TYPE"])["VALUE"]
    .sum()
    .reset_index()
    .pivot(index="EVSE", columns="TYPE", values="VALUE")
    .reset_index()
)

# print(f"\nEVSE_charge_tot:\n{df_evse_charge_results_tot}")

# DataFrame formatting
dfEVSE = df_evse_cs_results.merge(df_evse_charge_results_tot, on="EVSE", how="inner")
dfEVSE.drop(columns=["POWR"], inplace=True)
dfEVSE["ENG0"] = dfEVSE["ENG0"].apply(lambda x: round(x, 2))
dfEVSE["ENG+"] = dfEVSE["ENG+"].apply(lambda x: round(x, 2))
dfEVSE["COST"] = dfEVSE["COST"].apply(lambda x: round(x, 2))
dfEVSE = dfEVSE.reindex(
    columns=[
        "EVSE",
        "STAY",
        "SC%",
        "DC%",
        "DC",
        "RC",
        "FC",
        "CS",
        "ENG0",
        "ENG+",
        "COST",
    ]
)

# Aggregated results per TIME Slot
df_time_charge_results_tot = (
    df_evse_charge_results_melted.groupby(["TIME", "TYPE"])["VALUE"]
    .sum()
    .reset_index()
    .pivot(index="TIME", columns="TYPE", values="VALUE")
    .reset_index()
)

# DataFrame formatting
dfTIME = df_time_charge_results_tot
dfTIME["PRICE"] = EnergyPrice.round(2)
dfTIME["POWR"] = dfTIME["POWR"].apply(lambda x: round(x, 2))
dfTIME["ENG+"] = dfTIME["ENG+"].apply(lambda x: round(x, 2))
dfTIME["COST"] = dfTIME["COST"].apply(lambda x: round(x, 2))
dfTIME["PWR%"] = dfTIME["POWR"].apply(lambda x: round(100 * x / EX_MPO, 2))
dfTIME = dfTIME.reindex(
    columns=[
        "TIME",
        "PRICE",
        "COST",
        "ENG+",
        "POWR",
        "PWR%",
    ]
)

# print(f"\nTIME_charge_tot:\n{df_time_charge_results_tot}")


EVSE_CS:
      EVSE  STAY    SC%    DC%     DC     RC     FC      CS       ENG0
0   EVSE01  3.62  19.01  83.92  58.74  38.66  38.66  100.00  13.310000
1   EVSE02  5.19   3.12  49.36  34.55  34.55  29.07   84.14   2.184261
2   EVSE03  1.41  17.32  76.07  53.25  21.97  21.97  100.00  12.126466
3   EVSE04  5.96   0.41  98.19  68.74  41.98  13.82   32.91   0.288183
4   EVSE05  6.83   4.25  50.91  35.64  35.64  34.33   96.33   2.972748
5   EVSE06  2.28   6.08  71.49  50.04  20.25  20.25  100.00   4.259391
6   EVSE07  4.02   5.82  76.71  53.70  32.24  32.24  100.00   4.077208
7   EVSE08  1.98   5.84  61.98  43.39  17.93  17.93  100.00   4.090025
8   EVSE09  4.19  15.70  51.98  36.39  36.39  36.39  100.00  10.992463
9   EVSE10  4.60  11.85  42.79  29.95  29.95  29.95  100.00   8.293804
10  EVSE11  5.25   3.41  43.90  30.73  30.73  30.73  100.00   2.387338
11  EVSE12  7.64  19.31  88.50  61.95  61.95  31.52   50.87  13.518848
12  EVSE13  3.13   1.95  81.05  56.74  23.29  23.29  100.00   1.367

### Print

In [9]:
# print the objective outcome
print(f"\nObjective Outcome = {round(pyo.value(tgc.OBJ), 4)}")

print(f"\nEVSE Results\n\n{dfEVSE}\n")
# print(f"\nEVSE Total\nCOST: {round(dfEVSE['COST'].sum(),2)}\nENG+: {round(dfEVSE['ENG+'].sum(),2)}")

print(f"\nTIME Results\n\n{dfTIME}\n")
# print(f"\nTIME Total\nCOST: {round(dfTIME['COST'].sum(),2)}\nENG+: {round(dfTIME['ENG+'].sum(),2)}")

print(
    f"""\n \
    Totals\n\n \
    TIME: {round(sum(pt),2)} hrs\n \
    COST: € {round(dfTIME['COST'].sum(),2)}\n \
    ENG+: {round(dfTIME['ENG+'].sum(),2)} kWh\n \
    CSAT: {round(dfEVSE['CS'].mean(), 2)} %\n """
)


Objective Outcome = 1.1794

EVSE Results

      EVSE  STAY    SC%    DC%     DC     RC     FC      CS   ENG0   ENG+  \
0   EVSE01  3.62  19.01  83.92  58.74  38.66  38.66  100.00  13.31  25.35   
1   EVSE02  5.19   3.12  49.36  34.55  34.55  29.07   84.14   2.18  26.89   
2   EVSE03  1.41  17.32  76.07  53.25  21.97  21.97  100.00  12.13   9.85   
3   EVSE04  5.96   0.41  98.19  68.74  41.98  13.82   32.91   0.29  13.53   
4   EVSE05  6.83   4.25  50.91  35.64  35.64  34.33   96.33   2.97  31.36   
5   EVSE06  2.28   6.08  71.49  50.04  20.25  20.25  100.00   4.26  15.99   
6   EVSE07  4.02   5.82  76.71  53.70  32.24  32.24  100.00   4.08  28.17   
7   EVSE08  1.98   5.84  61.98  43.39  17.93  17.93  100.00   4.09  13.84   
8   EVSE09  4.19  15.70  51.98  36.39  36.39  36.39  100.00  10.99  25.39   
9   EVSE10  4.60  11.85  42.79  29.95  29.95  29.95  100.00   8.29  21.66   
10  EVSE11  5.25   3.41  43.90  30.73  30.73  30.73  100.00   2.39  28.34   
11  EVSE12  7.64  19.31  88.50  6

### Export